In [125]:
import numpy as np
import pandas as pd
import os
from urllib.parse import quote, unquote

In [104]:
path = '/home/ctang/ttmp/MMCSR/GiantMIDI'
file_path = path + '/full_music_pieces_youtube_similarity_pianosoloprob_split.csv'
GiantMidi = pd.read_csv(file_path)

In [105]:
# keep only relevant columns

GiantMidi = GiantMidi[['surname', 'firstname', 'music']]

# create new empty columns called 'midi_path' and 'bootleg_path'

GiantMidi['midi_path'] = ''
GiantMidi['bootleg_path'] = ''

In [108]:
# drop duplicate music with the same surname and firstname

GiantMidi = GiantMidi.drop_duplicates(subset=['surname', 'firstname', 'music'])

In [118]:
GiantMidi_copy = GiantMidi.copy()

In [111]:
midi_path = path + '/midis'

# for each row in GiantMidi, check if the midi file exists in the midi_path. If yes, add the path to the midi file to the dataframe

for row in GiantMidi.itertuples():
    header = str(row.surname) + ', ' + str(row.firstname) + ', ' + str(row.music)
    
    # check if the midi file starting with the header exists in the midi_path
    for file in os.listdir(midi_path):
        if file.startswith(header):
            GiantMidi.at[row.Index, 'midi_path'] = midi_path + '/' + file
            break

In [119]:
GiantMidi = GiantMidi[GiantMidi['midi_path'] != '']
GiantMidi = GiantMidi.reset_index(drop=True)

In [121]:
# Define the directory path
directory_path = "/mnt/data0/Datasets/imslp/score_scrape/results/composer"

# Change the current working directory
os.chdir(directory_path)

In [141]:
Bach = directory_path + '/Bach,_Johann_Sebastian'
# find the folders in richardson.

os.listdir(Bach)

['Partita_in_G_major,_BWV_829_',
 'Liebster_Herr_Jesu,_wo_bleibst_du_so_lange%3F,_BWV_484_',
 'An_Wasserfl%C3%BCssen_Babylon,_BWV_267_',
 'Prelude_in_C_minor,_BWV_999_',
 'Selig!_wer_an_Jesum_denkt,_BWV_498_',
 'Wo_gehest_du_hin,_BWV_166_',
 'Gott,_wie_dein_Name,_so_ist_auch_dein_Ruhm,_BWV_171_',
 'Ein_feste_Burg_ist_unser_Gott,_BWV_80b_',
 'Cello_Suite_No.3_in_C_major,_BWV_1009_',
 'Capriccio_in_B-flat_major,_BWV_992_',
 'Canon_in_G_major,_BWV_1077_',
 'Invention_in_E_minor,_BWV_778_',
 'Vom_Himmel_kam_der_Engel_Schaar,_BWV_607_',
 'Organ_Concerto_in_A_minor,_BWV_593_',
 'Partita_in_D_major,_BWV_828_',
 'Vergn%C3%BCgte_Ruh,_beliebte_Seelenlust,_BWV_170_',
 'Ich_ruf_zu_dir_Herr_Jesu_Christ,_BWV_Anh.73_',
 'Christ,_der_du_bist_der_helle_Tag,_BWV_766_',
 'Invention_in_B_minor,_BWV_786_',
 'Erschallet,_ihr_Lieder,_BWV_172_',
 'Herr_Gott,_dich_loben_alle_wir,_BWV_Anh.31_',
 'Erw%C3%A4hlte_Pleissenstadt,_BWV_216a_',
 'Schwingt_freudig_euch_empor,_BWV_36c_',
 'Fugue_in_D_minor,_BWV_Anh.99_',

In [152]:
for row in GiantMidi.itertuples():
    composer_directory = quote(str(row.surname)) + ",_" + quote(str(row.firstname)).replace("%20", "_")
    path = directory_path + '/' + composer_directory
    if os.path.exists(path):
        pieces = os.listdir(path)
        # transform the piece names
        for piece in pieces:
            try:
                piecename = piece.split(",")[0].replace("_", " ")
            except:
                piecename = piece.replace("_", " ")
            piecename = unquote(piecename)
            if row.music in piecename:
                GiantMidi.at[row.Index, 'bootleg_path'] = path + '/' + piece
                break

In [157]:
# replace the empty strings with NaN
GiantMidi = GiantMidi.replace('', np.nan)

# save the dataframe to a csv file
path = '/home/ctang/ttmp/MMCSR/GiantMIDI'
GiantMidi.to_csv(path + '/GiantMidi.csv', index=False)

In [160]:
GiantMidi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10800 entries, 0 to 10799
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   surname       10800 non-null  object
 1   firstname     10800 non-null  object
 2   music         10800 non-null  object
 3   midi_path     10800 non-null  object
 4   bootleg_path  3515 non-null   object
dtypes: object(5)
memory usage: 422.0+ KB
